In [ ]:
import pennylane as qml
from pennylane import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from tensorflow.keras.datasets import mnist
import time

# 1. Load and preprocess MNIST dataset
(X_train_full, y_train_full), (X_test_full, y_test_full) = mnist.load_data()

# Filter for digits 0 and 1 (binary classification)
binary_classes = [0, 1]
mask_train = np.isin(y_train_full, binary_classes)
mask_test = np.isin(y_test_full, binary_classes)
X_train = X_train_full[mask_train].reshape(-1, 784)  # Flatten 28x28 images to 784 features
y_train = y_train_full[mask_train]
X_test = X_test_full[mask_test].reshape(-1, 784)
y_test = y_test_full[mask_test]

# Reduce dataset size for quantum kernel feasibility (100 training, 20 test)
n_train = 50000
n_test = 20000
indices_train = np.random.permutation(len(X_train))[:n_train]
indices_test = np.random.permutation(len(X_test))[:n_test]
X_train, y_train = X_train[indices_train], y_train[indices_train]
X_test, y_test = X_test[indices_test], y_test[indices_test]

# Apply PCA to reduce to 2 features (for 2-qubit circuit)
n_features = 2
pca = PCA(n_components=n_features)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Normalize features
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Define the device (simulator; can use qiskit.aer with pennylane-qiskit)
dev = qml.device('default.qubit', wires=n_features)

# 3. Define the ZZFeatureMap equivalent
def feature_map(x, wires):
    for i in range(len(wires)):
        qml.RY(2 * np.pi * x[i], wires=wires[i])  # Rotation based on features
    qml.CNOT(wires=[wires[0], wires[1]])  # Entangling gate

# 4. Define the quantum kernel function
@qml.qnode(dev)
def kernel_circuit(x1, x2):
    feature_map(x1, wires=range(n_features))
    qml.adjoint(feature_map)(x2, wires=range(n_features))
    return qml.probs(wires=range(n_features))

def quantum_kernel(x1, x2):
    return kernel_circuit(x1, x2)[0]  # Fidelity (probability of |0...0>)

# 5. Compute the kernel matrix for training data
n_samples = len(X_train_scaled)
kernel_matrix = np.zeros((n_samples, n_samples))
print("Computing training kernel matrix...")
start_time = time.time()
for i in range(n_samples):
    for j in range(n_samples):
        kernel_matrix[i, j] = quantum_kernel(X_train_scaled[i], X_train_scaled[j])
print(f"Training kernel matrix computed in {time.time() - start_time:.2f} seconds")

# 6. Train the QSVC
qsvc = SVC(kernel='precomputed')
qsvc.fit(kernel_matrix, y_train)

# 7. Compute kernel matrix for test data
test_kernel_matrix = np.zeros((len(X_test_scaled), n_samples))
print("Computing test kernel matrix...")
start_time = time.time()
for i in range(len(X_test_scaled)):
    for j in range(n_samples):
        test_kernel_matrix[i, j] = quantum_kernel(X_test_scaled[i], X_train_scaled[j])
print(f"Test kernel matrix computed in {time.time() - start_time:.2f} seconds")

# 8. Make predictions and evaluate
y_pred = qsvc.predict(test_kernel_matrix)
accuracy = accuracy_score(y_test, y_pred)
print("Predictions:", y_pred)
print("True labels:", y_test)
print(f"Test accuracy: {accuracy:.4f}")

Computing training kernel matrix...
